In [22]:
import boto3
boto_client = boto3.setup_default_session(region_name='eu-west-1')

In [23]:
def create_MoreLikeThis_table(dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.create_table(
        TableName='MoreLikeThis',
        KeySchema=[
            {
                'AttributeName': 'item_id',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'item_id',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table

In [24]:
MoreLikeThis_table = create_MoreLikeThis_table()

In [4]:
def put_recs(item_id, recs_list, type_list, score_list, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('MoreLikeThis')
    response = table.put_item(
       Item={
           'item_id': item_id,
           'item_recs': recs_list,
           'item_type' : type_list,
           'item_score' : score_list
        }
    )
    return response

In [18]:
recs_resp = put_recs(
    'movida_52859',
    ['BBJ2550316A'] * 20 ,
    ['PROGRAM'] * 20,
    list(reversed(range(20)))
)

In [6]:
from botocore.exceptions import ClientError

In [11]:
def get_recs(item_id, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('MoreLikeThis')

    try:
        response = table.get_item(Key={'item_id': item_id})
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        return response['Item']

In [19]:
rec = get_recs('movida_52859')
if rec:
    print("Get rec succeeded:")
    print(rec)

Get rec succeeded:
{'item_score': [Decimal('19'), Decimal('18'), Decimal('17'), Decimal('16'), Decimal('15'), Decimal('14'), Decimal('13'), Decimal('12'), Decimal('11'), Decimal('10'), Decimal('9'), Decimal('8'), Decimal('7'), Decimal('6'), Decimal('5'), Decimal('4'), Decimal('3'), Decimal('2'), Decimal('1'), Decimal('0')], 'item_id': 'movida_52859', 'item_recs': ['BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A', 'BBJ2550316A'], 'item_type': ['PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM']}


In [23]:
from decimal import Decimal

In [15]:
def update_recs(item_id, recs_list, type_list, score_list, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('MoreLikeThis')

    response = table.update_item(
        Key={
            'item_id': item_id,
            },
        UpdateExpression="set item_recs=:r, item_type=:t, item_score=:s",
        ExpressionAttributeValues={
            ':r': recs_list,
            ':t': type_list,
            ':s': score_list
        },
        ReturnValues="UPDATED_NEW"
    )
    return response

In [17]:
update_response = update_recs(
    'movida_52859',
    ['BBJ2550316B'] * 20 ,
    ['PROGRAM'] * 20,
    list(reversed(range(20)))
)
print("Update recs succeeded:")
print(update_response)
print(update_response['Attributes']['item_recs'])

Update recs succeeded:
{'Attributes': {'item_score': [Decimal('19'), Decimal('18'), Decimal('17'), Decimal('16'), Decimal('15'), Decimal('14'), Decimal('13'), Decimal('12'), Decimal('11'), Decimal('10'), Decimal('9'), Decimal('8'), Decimal('7'), Decimal('6'), Decimal('5'), Decimal('4'), Decimal('3'), Decimal('2'), Decimal('1'), Decimal('0')], 'item_recs': ['BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B', 'BBJ2550316B'], 'item_type': ['PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM', 'PROGRAM']}, 'ResponseMetadata': {'RequestId': 'E0G0ICCVBE9MI9HIHEESACHC0VVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCod